<h3>
XIRR (Extended Internal Rate of Return) - Mutual Fund Investments
</h3>

In [17]:
import numpy as np
import pandas as pd
from pyxirr import xirr
from datetime import date, datetime as dt
from scipy.optimize import newton

In [32]:
mutual_fund = pd.read_csv(r'mutual_fund_xirr.csv')

In [33]:
mutual_fund

,Date,Cashflow,Type
0,01-01-2015,-5000,SIP
1,01-02-2015,-5000,SIP
2,01-03-2015,-5000,SIP
3,01-04-2015,-5000,SIP
4,01-05-2015,-5000,SIP
5,25-05-2015,10000,Redemption
6,01-06-2015,-5000,SIP
7,01-07-2015,-5000,SIP
8,01-08-2015,5000,SWP
9,01-09-2015,5000,SWP


In [20]:
mutual_fund.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      14 non-null     object
 1   Cashflow  14 non-null     int64 
 2   Type      14 non-null     object
dtypes: int64(1), object(2)
memory usage: 464.0+ bytes


In [21]:
mutual_fund['Date'] = [ dt.strptime(d, '%d-%m-%Y')   for d in mutual_fund.Date]

In [22]:
mutual_fund.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      14 non-null     datetime64[ns]
 1   Cashflow  14 non-null     int64         
 2   Type      14 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 464.0+ bytes


In [25]:
first_date = mutual_fund.loc[0].Date
first_date

Timestamp('2015-01-01 00:00:00')

In [26]:
mutual_fund['days_elapsed'] = [(i-first_date).days for i in mutual_fund['Date'].values]

In [27]:
mutual_fund

,Date,Cashflow,Type,days_elapsed
0,2015-01-01,-5000,SIP,0
1,2015-02-01,-5000,SIP,31
2,2015-03-01,-5000,SIP,59
3,2015-04-01,-5000,SIP,90
4,2015-05-01,-5000,SIP,120
5,2015-05-25,10000,Redemption,144
6,2015-06-01,-5000,SIP,151
7,2015-07-01,-5000,SIP,181
8,2015-08-01,5000,SWP,212
9,2015-09-01,5000,SWP,243


In [28]:
discount_list = []
npv_list = []

for rate in np.arange(0.3, .4, .000001) :
    npv = sum([cf/(1+rate)**(d/365) for d, cf in zip(mutual_fund.days_elapsed, mutual_fund.Cashflow)])
    discount_list.append(rate)
    npv_list.append(npv)
    if npv < 0.001 :
        break

In [29]:
list(zip(discount_list[-1::-1], npv_list[-1::-1]))[:5:1]

[(0.3597889999984003, -0.003666210115170543),
 (0.35978799999840033, 0.006662086197138706),
 (0.35978699999840036, 0.016990397266454238),
 (0.3597859999984004, 0.027318723085500096),
 (0.3597849999984004, 0.037647063650183554)]

In [30]:
xirr(mutual_fund.Date, mutual_fund.Cashflow)

0.35978864503068003

In [31]:
def get_XIRR(rate) :
    return sum([cf/(1+rate)**(days/365) for days, cf in zip(mutual_fund.days_elapsed, mutual_fund.Cashflow)])

newton(get_XIRR, .1)

0.3597886450306787